In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import openpyxl
import os
import pandas as pd
from dataclasses import dataclass
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from typing import List
import json
from dataclasses import asdict

In [ ]:
def get_driver(url):
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--window-size=1366,768")
    options.add_argument("--user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36")
    driver = webdriver.Chrome(options=options)
    try:
        driver.maximize_window()
    except Exception:
        pass
    driver.get(url)
    return driver

In [63]:
def get_workbook(file_name):
    if os.path.exists(file_name):
        workbook = openpyxl.load_workbook(file_name)
    else:
        workbook = openpyxl.Workbook()
        workbook.save(file_name)

    return workbook

In [64]:
def parse_min_max(text):
    result = {}

    # Min değeri
    min_match = re.search(r'Min[:.]?\s*([\d.,KLkl]+)', text)
    if min_match:
        result['Min'] = min_match.group(1)

    # Max değeri
    max_match = re.search(r'Max[:.]?\s*([\d.,KLkl]+)', text)
    if max_match:
        result['Max'] = max_match.group(1)

    return result

In [65]:
def get_market_type_2(driver, match_link_param, match_name_param, is_mac_active):
    @dataclass
    class BackJsonDTO:
        price: str
        size: str

    @dataclass
    class RunnerJsonDTO:
        name: str
        status: str
        min: str
        max: str
        backs: List[BackJsonDTO]
        lays: List[BackJsonDTO]

    @dataclass
    class DetailJsonDTO:
        sport_name: str
        match_name: str
        match_link: str
        market_name: str
        is_active: bool
        market_type: str
        runners: List[RunnerJsonDTO]

    market_games_market_2 = driver.find_elements(By.CSS_SELECTOR, "div.game-market.market-2")
    if len(market_games_market_2) > 0:
        for game in market_games_market_2:

            try:
                market_title_class_dv = game.find_element(By.CLASS_NAME, "market-title")
                market_title_dv = market_title_class_dv.find_element(By.TAG_NAME, "span").text
                min_max_dv = game.find_element(By.CLASS_NAME, "market-nation-name").text

                market_2_main_class = game.find_element(By.CLASS_NAME, "market-body")
                
                market_2_rows = market_2_main_class.find_elements(By.CLASS_NAME, "market-row")

                runners_market_2 : List[RunnerJsonDTO] = []


                for market_row in market_2_rows:
                    attr_dv = market_row.get_attribute("data-title") if market_row.get_attribute("data-title") else 'Active'
                    name_dv = market_row.find_element(By.CLASS_NAME, "market-nation-name").text
        

                    back_div = market_row.find_elements(By.CSS_SELECTOR, "div.market-odd-box.back")
                    back_spans = back_div[0].find_elements(By.TAG_NAME, "span") if back_div else []
                    back_dv_price = back_spans[0].text if len(back_spans) > 0 else None
                    back_dv_size = back_spans[1].text if len(back_spans) > 1 else None


                    lay_div = market_row.find_elements(By.CSS_SELECTOR, "div.market-odd-box.lay")
                    lay_spans = lay_div[0].find_elements(By.TAG_NAME, "span") if lay_div else []
                    lay_dv_price = lay_spans[0].text if len(lay_spans) > 0 else None
                    lay_dv_size = lay_spans[1].text if len(lay_spans) > 1 else None

                    runners_market_2.append(RunnerJsonDTO(
                        name=name_dv,
                        status=attr_dv,
                        min= parse_min_max(min_max_dv).get('Min', None),
                        max= parse_min_max(min_max_dv).get('Max', None),
                        backs=[
                            BackJsonDTO(price=back_dv_price, size=back_dv_size),
                        ],
                        lays=[
                            BackJsonDTO(price=lay_dv_price, size=lay_dv_size),
                        ]
                    ))


                match_info = DetailJsonDTO( sport_name= 'Cricket',
                                            match_name=name_dv,
                                            match_link=match_link_param,
                                            market_name=market_title_dv,
                                            is_active=is_mac_active == 'True',
                                            market_type='market-2',
                                            runners=runners_market_2
                                        )

                # Dataclass'ı dict'e çevir
                match_info_dict = asdict(match_info)

                # JSON olarak yazdır / stringe çevir
                match_info_json = json.dumps(match_info_dict, indent=4)

                print(match_info_json)
        
            except Exception as e:
                print('market 2, ' + str(e))
    

In [66]:
def get_market_type_4(driver, match_link_param, match_name_param, is_mac_active):
    @dataclass
    class BackJsonDTO:
        price: str
        size: str

    @dataclass
    class RunnerJsonDTO:
        name: str
        status: str
        min: str
        max: str
        backs: List[BackJsonDTO]
        lays: List[BackJsonDTO]

    @dataclass
    class DetailJsonDTO:
        sport_name: str
        match_name: str
        match_link: str
        market_name: str
        is_active: bool
        market_type: str
        runners: List[RunnerJsonDTO]

    market_games_market_4 = driver.find_elements(By.CSS_SELECTOR, "div.game-market.market-4")
    if len(market_games_market_4) > 0:
        for game in market_games_market_4:

            try:
                market_title_class_dv = game.find_element(By.CLASS_NAME, "market-title")
                market_title_dv = market_title_class_dv.find_element(By.TAG_NAME, "span").text
                min_max_dv = game.find_element(By.CLASS_NAME, "market-nation-name").text

                market_4_main_class = game.find_element(By.CLASS_NAME, "market-body")
                market_4_rows = market_4_main_class.find_elements(By.CLASS_NAME, "market-row")

                runners_market_4 : List[RunnerJsonDTO] = []


                for market_row in market_4_rows:
                    attr_dv = market_row.get_attribute("data-title") if market_row.get_attribute("data-title") else 'Active'
                    name_dv = market_row.find_element(By.CLASS_NAME, "market-nation-name").text
                    back2_div = market_row.find_element(By.CSS_SELECTOR, "div.market-odd-box.back2")
                    back2_spans = back2_div.find_elements(By.TAG_NAME, "span")
                    back2_dv_price = back2_spans[0].text if len(back2_spans) > 0 else None
                    back2_dv_size  = back2_spans[1].text if len(back2_spans) > 1 else None

                    back1_div = market_row.find_element(By.CSS_SELECTOR, "div.market-odd-box.back1")
                    back1_spans = back1_div.find_elements(By.TAG_NAME, "span")
                    back1_dv_price = back1_spans[0].text if len(back1_spans) > 0 else None
                    back1_dv_size = back1_spans[1].text if len(back1_spans) > 1 else None

                    back_div = market_row.find_element(By.CSS_SELECTOR, "div.market-odd-box.back")
                    back_spans = back_div.find_elements(By.TAG_NAME, "span")    
                    back_dv_price = back_spans[0].text if len(back_spans) > 0 else None
                    back_dv_size = back_spans[1].text if len(back_spans) > 1 else None

                    lay2_div = market_row.find_element(By.CSS_SELECTOR, "div.market-odd-box.lay2")
                    lay2_spans = lay2_div.find_elements(By.TAG_NAME, "span")
                    lay2_dv_price = lay2_spans[0].text if len(lay2_spans) > 0 else None
                    lay2_dv_size = lay2_spans[1].text if len(lay2_spans) > 1 else None

                    lay1_div = market_row.find_element(By.CSS_SELECTOR, "div.market-odd-box.lay1")
                    lay1_spans = lay1_div.find_elements(By.TAG_NAME, "span")
                    lay1_dv_price = lay1_spans[0].text if len(lay1_spans) > 0 else None
                    lay1_dv_size = lay1_spans[1].text if len(lay1_spans) > 1 else None

                    lay_div = market_row.find_element(By.CSS_SELECTOR, "div.market-odd-box.lay")
                    lay_spans = lay_div.find_elements(By.TAG_NAME, "span")
                    lay_dv_price = lay_spans[0].text if len(lay_spans) > 0 else None
                    lay_dv_size = lay_spans[1].text if len(lay_spans) > 1 else None

                    runners_market_4.append(RunnerJsonDTO(
                        name=name_dv,
                        status=attr_dv,
                        min= parse_min_max(min_max_dv).get('Min', None),
                        max= parse_min_max(min_max_dv).get('Max', None),
                        backs=[
                            BackJsonDTO(price=back1_dv_price, size=back1_dv_size),
                            BackJsonDTO(price=back2_dv_price, size=back2_dv_size),
                            BackJsonDTO(price=back_dv_price, size=back_dv_size),
                        ],
                        lays=[
                            BackJsonDTO(price=lay1_dv_price, size=lay1_dv_size),
                            BackJsonDTO(price=lay2_dv_price, size=lay2_dv_size),
                            BackJsonDTO(price=lay_dv_price, size=lay_dv_size),
                        ]
                    ))


                match_info = DetailJsonDTO( sport_name= 'Cricket',
                                            match_name=name_dv,
                                            match_link=match_link_param,
                                            market_name=market_title_dv,
                                            is_active=is_mac_active == 'True',
                                            market_type='market-4',
                                            runners=runners_market_4
                                        )

                # Dataclass'ı dict'e çevir
                match_info_dict = asdict(match_info)

                # JSON olarak yazdır / stringe çevir
                match_info_json = json.dumps(match_info_dict, indent=4)

                print(match_info_json)
        
            except Exception as e:
                print('market 4, ' + str(e))


In [67]:
def get_market_type_6(driver, match_link_param, match_name_param, is_mac_active):
    @dataclass
    class YesNoJsonDto:
        price: str
        size: str

    @dataclass
    class RunnerJsonDTO:
        name: str
        status: str
        min: str
        max: str
        YesList: List[YesNoJsonDto]
        NoList: List[YesNoJsonDto]

    @dataclass
    class DetailJsonDTO:
        sport_name: str
        match_name: str
        match_link: str
        market_name: str
        is_active: bool
        market_type: str
        runners: List[RunnerJsonDTO]

    market_games_market_6 = driver.find_elements(By.CSS_SELECTOR, "div.game-market.market-6")
    if len(market_games_market_6) > 0:
        for game in market_games_market_6:

            try:
                market_title_class_dv = game.find_element(By.CLASS_NAME, "market-title")
                market_title_dv = market_title_class_dv.find_element(By.TAG_NAME, "span").text
                min_max_dv = game.find_element(By.CLASS_NAME, "market-nation-name").text

                market_6_main_class = game.find_element(By.CLASS_NAME, "market-body")
                market_6_rows = market_6_main_class.find_elements(By.CLASS_NAME, "market-row")
              
                runners_market_6 : List[RunnerJsonDTO] = []


                for market_row in market_6_rows:
                    attr_dv = market_row.get_attribute("data-title") if market_row.get_attribute("data-title") else 'Active'
                    name_dv = market_row.find_element(By.CLASS_NAME, "market-nation-name").text
                    
                    back_div = market_row.find_elements(By.CSS_SELECTOR, "div.market-odd-box.back")
                    back_spans = back_div[0].find_elements(By.TAG_NAME, "span") if back_div else []
                    back_dv_price = back_spans[0].text if len(back_spans) > 0 else None
                    back_dv_size = back_spans[1].text if len(back_spans) > 1 else None


                    lay_div = market_row.find_elements(By.CSS_SELECTOR, "div.market-odd-box.lay")
                    lay_spans = lay_div[0].find_elements(By.TAG_NAME, "span") if lay_div else []
                    lay_dv_price = lay_spans[0].text if len(lay_spans) > 0 else None
                    lay_dv_size = lay_spans[1].text if len(lay_spans) > 1 else None

                    runners_market_6.append(RunnerJsonDTO(
                        name=name_dv,
                        status=attr_dv,
                        min= parse_min_max(min_max_dv).get('Min', None),
                        max= parse_min_max(min_max_dv).get('Max', None),
                        YesList=[
                            YesNoJsonDto(price=back_dv_price, size=back_dv_size),
                        ],
                        NoList=[
                            YesNoJsonDto(price=lay_dv_price, size=lay_dv_size),
                        ]
                    ))


                match_info = DetailJsonDTO( sport_name= 'Cricket',
                                            match_name=name_dv,
                                            match_link=match_link_param,
                                            market_name=market_title_dv,
                                            is_active=is_mac_active == 'True',
                                            market_type='market-6',
                                            runners=runners_market_6
                                        )

                # Dataclass'ı dict'e çevir
                match_info_dict = asdict(match_info)

                # JSON olarak yazdır / stringe çevir
                match_info_json = json.dumps(match_info_dict, indent=4)

                print(match_info_json)
        
            except Exception as e:
                print('market 6, ' + str(e))


In [68]:
def get_market_type_9(driver, match_link_param, match_name_param, is_mac_active):
    @dataclass
    class BackJsonDTO:
        price: str
        size: str

    @dataclass
    class RunnerJsonDTO:
        name: str
        status: str
        min: str
        max: str
        Backs: List[BackJsonDTO]
        Lays: List[BackJsonDTO]

    @dataclass
    class DetailJsonDTO:
        sport_name: str
        match_name: str
        match_link: str
        market_name: str
        is_active: bool
        market_type: str
        runners: List[RunnerJsonDTO]

    market_games_market_9 = driver.find_elements(By.CSS_SELECTOR, "div.game-market.market-9")
    if len(market_games_market_9) > 0:
        for game in market_games_market_9:

            try:
                market_title_class_dv = game.find_element(By.CLASS_NAME, "market-title")
                market_title_dv = market_title_class_dv.find_element(By.TAG_NAME, "span").text
                min_max_dv = game.find_element(By.CLASS_NAME, "market-nation-name").text

                market_9_main_class = game.find_element(By.CLASS_NAME, "market-body")
                market_9_rows = market_9_main_class.find_elements(By.CLASS_NAME, "market-row")
              
                runners_market_9 : List[RunnerJsonDTO] = []


                for market_row in market_9_rows:
                    attr_dv = market_row.get_attribute("data-title") if market_row.get_attribute("data-title") else 'Active'
                    name_dv = market_row.find_element(By.CLASS_NAME, "market-nation-name").text
                    
                    back_div = market_row.find_elements(By.CSS_SELECTOR, "div.market-odd-box.back")
                    back_spans = back_div[0].find_elements(By.TAG_NAME, "span") if back_div else []
                    back_dv_price = back_spans[0].text if len(back_spans) > 0 else None
                    back_dv_size = back_spans[1].text if len(back_spans) > 1 else None


                    lay_div = market_row.find_elements(By.CSS_SELECTOR, "div.market-odd-box.lay")
                    lay_spans = lay_div[0].find_elements(By.TAG_NAME, "span") if lay_div else []
                    lay_dv_price = lay_spans[0].text if len(lay_spans) > 0 else None
                    lay_dv_size = lay_spans[1].text if len(lay_spans) > 1 else None

                    runners_market_9.append(RunnerJsonDTO(
                        name=name_dv,
                        status=attr_dv,
                        min= parse_min_max(min_max_dv).get('Min', None),
                        max= parse_min_max(min_max_dv).get('Max', None),
                        YesList=[
                            BackJsonDTO(price=back_dv_price, size=back_dv_size),
                        ],
                        NoList=[
                            BackJsonDTO(price=lay_dv_price, size=lay_dv_size),
                        ]
                    ))


                match_info = DetailJsonDTO( sport_name= 'Cricket',
                                            match_name=name_dv,
                                            match_link=match_link_param,
                                            market_name=market_title_dv,
                                            is_active=is_mac_active == 'True',
                                            market_type='market-9',
                                            runners=runners_market_9
                                        )

                # Dataclass'ı dict'e çevir
                match_info_dict = asdict(match_info)

                # JSON olarak yazdır / stringe çevir
                match_info_json = json.dumps(match_info_dict, indent=4)

                print(match_info_json)
        
            except Exception as e:
                print('market 6, ' + str(e))


In [69]:
def get_market_type_10(driver, match_link_param, match_name_param, is_mac_active):
    @dataclass
    class YesNoJsonDto:
        price: str
        size: str

    @dataclass
    class RunnerJsonDTO:
        name: str
        status: str
        min: str
        max: str
        YesList: List[YesNoJsonDto]
        NoList: List[YesNoJsonDto]

    @dataclass
    class DetailJsonDTO:
        sport_name: str
        match_name: str
        match_link: str
        market_name: str
        is_active: bool
        market_type: str
        runners: List[RunnerJsonDTO]

    market_games_market_10 = driver.find_elements(By.CSS_SELECTOR, "div.game-market.market-10")

    if len(market_games_market_10) > 0:
        for game in market_games_market_10:

            try:
                
                market_title_class_dv = game.find_element(By.CLASS_NAME, "market-title")
                market_title_dv = market_title_class_dv.find_element(By.TAG_NAME, "span").text
                min_max_dv = game.find_element(By.CLASS_NAME, "market-nation-name").text

                market_10_main_class = game.find_element(By.CLASS_NAME, "market-body")
                market_10_rows = market_10_main_class.find_elements(By.CLASS_NAME, "market-row")
              
                runners_market_10 : List[RunnerJsonDTO] = []


                for market_row in market_10_rows:
                    attr_dv = market_row.get_attribute("data-title") if market_row.get_attribute("data-title") else 'Active'
                    name_dv = market_row.find_element(By.CLASS_NAME, "market-nation-name").text
                    
                    back_div = market_row.find_element(By.CSS_SELECTOR, "div.market-odd-box.back")
                    back_spans = back_div.find_elements(By.TAG_NAME, "span")    
                    back_dv_price = back_spans[0].text if len(back_spans) > 0 else None
                    back_dv_size = back_spans[1].text if len(back_spans) > 1 else None


                    lay_div = market_row.find_element(By.CSS_SELECTOR, "div.market-odd-box.lay")
                    lay_spans = lay_div.find_elements(By.TAG_NAME, "span")
                    lay_dv_price = lay_spans[0].text if len(lay_spans) > 0 else None
                    lay_dv_size = lay_spans[1].text if len(lay_spans) > 1 else None

                    runners_market_10.append(RunnerJsonDTO(
                        name=name_dv,
                        status=attr_dv,
                        min= parse_min_max(min_max_dv).get('Min', None),
                        max= parse_min_max(min_max_dv).get('Max', None),
                        YesList=[
                            YesNoJsonDto(price=back_dv_price, size=back_dv_size),
                        ],
                        NoList=[
                            YesNoJsonDto(price=lay_dv_price, size=lay_dv_size),
                        ]
                    ))


                match_info = DetailJsonDTO( sport_name= 'Cricket',
                                            match_name=name_dv,
                                            match_link=match_link_param,
                                            market_name=market_title_dv,
                                            is_active=is_mac_active == 'True',
                                            market_type='market-10',
                                            runners=runners_market_10
                                        )

                # Dataclass'ı dict'e çevir
                match_info_dict = asdict(match_info)

                # JSON olarak yazdır / stringe çevir
                match_info_json = json.dumps(match_info_dict, indent=4)

                print(match_info_json)
        
            except Exception as e:
                print('market 6, ' + str(e))


In [70]:

def get_all_market_type_data(driver,match_link_param,match_name_param,is_mac_active):

    
    driver.get(match_link_param)
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'game-market')))
    
    get_market_type_2(driver, match_link_param, match_name_param, is_mac_active)
    get_market_type_4(driver, match_link_param, match_name_param, is_mac_active)
    get_market_type_6(driver, match_link_param, match_name_param, is_mac_active)
    get_market_type_9(driver, match_link_param, match_name_param, is_mac_active)
    get_market_type_10(driver, match_link_param, match_name_param, is_mac_active)
    
    




In [ ]:
def get_sport_details_run():
    file_name = "./data/cricket-matches.xlsx"
    workbook = get_workbook(file_name)
    sheet = workbook.active

    driver = get_driver('https://d247.com/home')

    time.sleep(1)
    element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='root']/div[2]/div/div/div[2]/form/div[3]/button[2]")))
    element.click()                                                                  
    time.sleep(1)
    element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[3]/div/div/div[1]/button")))
    element.click()
    
    for row in sheet.iter_rows(min_row=2,max_row=5, values_only=True):
        try:
            match_link = row[1]
            match_name = row[0]
            is_mac_active = row[2]
            get_all_market_type_data(driver,match_link,match_name,is_mac_active)


        except Exception as e:
            print('error match : ' + match_link + ' ' + str(e))
        

    workbook.close()

    driver.quit()

In [72]:
get_sport_details_run()

{
    "sport_name": "Cricket",
    "match_name": "ENG",
    "match_link": "https://d247.com/casino/superover2",
    "market_name": "Bookmaker",
    "is_active": true,
    "market_type": "market-2",
    "runners": [
        {
            "name": "IND",
            "status": "SUSPENDED",
            "min": "100.00",
            "max": "3L",
            "backs": [
                {
                    "price": "-",
                    "size": "300000"
                }
            ],
            "lays": [
                {
                    "price": "-",
                    "size": "300000"
                }
            ]
        },
        {
            "name": "ENG",
            "status": "ACTIVE",
            "min": "100.00",
            "max": "3L",
            "backs": [
                {
                    "price": "1.32",
                    "size": "300000"
                }
            ],
            "lays": [
                {
                    "price": "1.38",
            